In [1]:
# math and robotics
import pathlib
import numpy as np
import spatialmath as sm
from comau_msgs.srv import SetIO, SetIORequest

# ROS
import actionlib
import rospy
import hrr_common
from hr_recycler_msgs.msg import (
    AdaptiveGraspingAction, AdaptiveGraspingGoal,
    CuttingAction, CuttingGoal,
    ChangeToolAction, ChangeToolGoal,    
    FingerGraspAction, FingerGraspGoal,    
    PushOpenAction, PushOpenGoal,
    VacuumGraspingAction, VacuumGraspingGoal,
    UnscrewAction, UnscrewGoal
)
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal

# set printing and plotting options
np.set_printoptions(precision=5, suppress=True)

from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType

from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common as hrr_cm

In [2]:
tool_change_pose = [-0.84518,  0.55554, -1.57835,  0.20987, -0.43471, -2.6285 ] #[-0.9376716018,  0.7174308896, -1.2232894897, -0.0096366024, -0.227122277 , -2.4522731304]

In [3]:
rospy.init_node('microwave_demo')

In [4]:
cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
change_tool_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}change_tool_action_srv_name")
calibration_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}calibration_action_srv_name")
cutting_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}cutting_action_srv_name")
grinding_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}grinding_action_srv_name")
pc_opening_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}pc_opening_action_srv_name")
sensitive_grasping_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}sensitive_grasping_action_srv_name")
unscrew_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}unscrew_action_srv_name")
vacuum_pick_place_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}vacuum_pick_place_action_srv_name")
finger_grasping_action_srv_name= hrr_common.ros_utils.get_param(f"{cobot_ns}finger_grasping_action_srv_name")
#pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)

def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client
def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()

def cancel(cl):
    cl.cancel_all_goals()
    
    
def set_cutting_goal(pointA, pointB):
    material= MaterialType()
    material = material.PLASTIC

    normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalB")
    normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")

    goal=CuttingGoal()
    goal.surface_normal_start=normalA
    goal.surface_normal_end=normalB
    goal.start_location=pointA
    goal.end_location=pointB
    goal.material.material_type=material
    goal.timeout=5000
    goal.thickness = 0.005 #not important. can be anything
    return goal
    
def set_screwing_goal(B_normal, B_screw_guess):
    sc_goal = UnscrewGoal()
    sc_goal.screw_location.header.frame_id = "hrr_cobot.base_link"
    sc_goal.surface_normal.header.frame_id = "hrr_cobot.base_link"
    sc_goal.surface_normal.vector = hrr_common.ros_utils.np2vec3(B_normal)
    sc_goal.screw_location.point = hrr_common.ros_utils.np2vec3(B_screw_guess)
    sc_goal.contact_force = 5
    sc_goal.insertion_force = 10
    sc_goal.timeout = 1200.0
    return sc_goal
    
    
vacuum_client = get_client(vacuum_pick_place_action_srv_name, VacuumGraspingAction)
pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)
shaftgrinder_client=get_client(cutting_action_srv_name, CuttingAction)
grasping_client = get_client(sensitive_grasping_action_srv_name, AdaptiveGraspingAction)
unscrewing_client = get_client(unscrew_action_srv_name, UnscrewAction)
calibrate_client = get_client(calibration_action_srv_name, CalibrateCobotAction)

### Load robot

In [5]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")
#cobot.change_tool("nothing")

[rosout] [/microwave_demo] Initialize MATLAB-Motion planner @ /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src
[rosout] [/microwave_demo] connected to Matlab
Shutting down global node /matlab_global_node_64741 with NodeURI http://192.168.2.87:44429/
The value of the ROS_IP environment variable, 192.168.2.87, will be used to set the advertised address for the ROS node.
Initializing global node /matlab_global_node_99632 with NodeURI http://192.168.2.87:43465/
[rosout] [/microwave_demo] MATLAB-Initialization done


pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


In [ ]:
cobot.change_tool("shaftgrinder")

In [6]:
cobot.tool

'shaft_grinder'

In [ ]:
#!rosparam get /hrr_cobot/tool_name

In [ ]:
cobot.B_F_msr

cobot.change_tool("shaftgrinder")
cobot.tool_controller.run_shaftgrinder(1.0,"slow")

# Microwave disassembly

## Tool Change to Screwdriver

In [ ]:
# cobot.move_to_joint_pose(tool_change_pose, stochastic=True)
# rospy.sleep(1)
# cobot.open_tool_changer()
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.SCREW_DRIVER
tool_change_client.send_goal(tc_goal)
# cancel(tool_change_client)

In [ ]:
cobot.close_tool_changer()


In [ ]:
cobot.change_tool("screwdriver")


## Calibrate screwdriver

In [ ]:
goal = CalibrateCobotGoal()
data_path = pathlib.Path.home() / "Documents" / "skill_recordings"
data_path.mkdir(parents=True, exist_ok=True)
tool_name = hrr_common.ros_utils.get_param(f'{cobot_ns}tool_name')
if tool_name == '':
    tool_name = 'nothing'
goal.calibration_file = str(data_path / f"{tool_name}.pkl")
goal.data_file = str(data_path / f"{tool_name}_calibration.npy")
goal.recalibrate = False
goal.scale_q4 = 0.5
goal.scale_q5 = 0.5
goal.scale_q6 = 0.5

In [ ]:
calibrate_client.send_goal(goal)

In [ ]:
cobot.update()
cobot.FT.read_params()
cobot.update()

In [ ]:
cobot.q

In [ ]:
cobot.T_B_E_robot.A

In [45]:
cobot.goTo(sm.SE3(0,0.00,0.15)@ cobot.T_B_E_robot)

In [ ]:
cobot.IK(cobot.T_B_E_robot)

In [ ]:
# cancel(unscrewing_client)

### Screw 1 - Front left of the microwave

In [ ]:
x: 0.525430364661
    y: -0.145763534197
    z: 0.342870911398
x: 0.0492269555456
    y: 0.0963132586081
    z: 0.99413302081

In [ ]:
B_normal = (np.r_[0.0492269555456,  0.0963132586081,  0.99413302081])
B_screw_guess = np.r_[ 0.525430364661-0.001+0.002,-0.145763534197+0.028+0.005-0.004, 0.342870911398]
sc_goal=set_screwing_goal(B_normal=B_normal, B_screw_guess=B_screw_guess)

In [ ]:
unscrewing_client.send_goal(sc_goal)

In [ ]:
cancel(unscrewing_client)

In [ ]:
cobot.T_B_C_robot

In [ ]:
cobot.change_tool("nothing")

### Screw 2 - Back right of the microwave

In [ ]:
B_normal = sm.base.rotx(-5, "deg") @ sm.base.unitvec(np.r_[0., 0.0, 1.0])
B_screw_guess = np.r_[0.47646+0.001, 0.2437+0.008 , 0.31]
sc_goal=set_screwing_goal(B_normal=B_normal, B_screw_guess=B_screw_guess)

In [ ]:
unscrewing_client.send_goal(sc_goal)

In [ ]:
cancel(unscrewing_client)

## Tool Change to Shaftgrinder

In [ ]:
# cobot.change_tool("nothing")
# cobot.move_to_joint_pose(tool_change_pose, stochastic=True)
# rospy.sleep(3)
# cobot.open_tool_changer()
# srv = rospy.ServiceProxy("/hrr_cobot/set_digital_io", SetIO)
# srv(SetIORequest(pin=14, state=False))
# srv(SetIORequest(pin=13, state=False))
# srv(SetIORequest(pin=12, state=False))
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.SHAFT_GRINDER
tool_change_client.send_goal(tc_goal)
# cancel(tool_change_client)

In [ ]:
cobot.open_tool_changer()

In [ ]:
cobot.change_tool("shaftgrinder")


In [ ]:
#tool_change_client.cancel_all_goals()

In [ ]:
cobot.close_tool_changer()

In [ ]:
cobot.q

In [ ]:
cobot.q

In [46]:
cobot.move_to_joint_pose([ 1.34109,  0.09328, -1.44435, -0.01627,  0.67686, -0.00009], stochastic=True)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 82)
In HrRecyclerPerecptioPipeline (line 51)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 82)
In HrRecyclerPerecptioPipeline (line 51)

calculating signed distance field ...
calculating signed distance field done
Elapsed time is 2.801787 seconds.
Elapsed time is 0.021384 seconds.


[rosout] [/microwave_demo] Planner is done and execution is running ...


## Cutting

In [ ]:
x: 0.699120541878
    y: -0.228496585032
    z: 0.225994310082
11:47 Uhr
: 0.699120541878
    y: -0.208496585032
    z: 0.225994310082

In [42]:
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.699120541878+0.015, -0.228496585032+0.035,  0.225994310082-0.005], "pointA")#[0.6921,0.2472,0.],"pointA")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.699120541878+0.015, -0.208496585032+0.04,  0.225994310082-0.005], "pointB")
goal = set_cutting_goal(pointA=pointA, pointB=pointB)

In [43]:
shaftgrinder_client.send_goal(goal)

In [ ]:
cobot.move_to_joint_pose([ 1.34109,  0.09328, -1.44435, -0.01627,  0.67686, -0.00009], stochastic=True)

In [44]:
cancel(shaftgrinder_client)

In [ ]:
cobot.change_tool("nothing")


In [ ]:
srv = rospy.ServiceProxy("/hrr_cobot/set_digital_io", SetIO)
srv(SetIORequest(pin=14, state=False))
srv(SetIORequest(pin=13, state=False))
srv(SetIORequest(pin=12, state=False))

In [ ]:
cobot.move_to_joint_pose(tool_change_pose, stochastic=True)


## Tool changing to nothing

In [ ]:
# cobot.move_to_joint_pose(tool_change_pose, stochastic=True)
# rospy.sleep(3)
# cobot.open_tool_changer()
# rospy.sleep(3)
# cobot.close_tool_changer()
# cobot.change_tool("nothing")
# # tc_goal = ChangeToolGoal()
# #tc_goal.new_tool.type = tc_goal.new_tool.WSG_50
# # tool_change_client.send_goal(tc_goal)
# # tool_change_client.cancel_all_goals()